In [1]:
import pandas as pd
import re

In [4]:
bank_statement_df = pd.read_excel(r"D:\NPS_phase1\nps-reconciliation\output\BANK STMT PHASE 4\Bank_Green.xlsx",skiprows=1)

In [5]:
bank_statement_df.head(7)

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number
0,1,946402601,500.00,CR,9866468505,100003988014,453362,2023-08-29T09:50:00,NaN
1,2,946402601,-67.83,DR,shrawancommi-080,shrawancommi-080,IntS100000029254,2023-08-29T17:28:00,NaN
2,3,946402601,-4.42,DR,asharcommi2080(16th,asharcommi2080(16th,IntS100000029253,2023-08-29T17:27:00,NaN


In [11]:
bank_statement_df['Desc3'].astype(str)

0              453362
1    IntS100000029254
2    IntS100000029253
Name: Desc3, dtype: object

In [29]:
for index, row in bank_statement_df.iterrows():
    if "10000" in str(row['Desc2']):
        id = re.findall(r'10*[0-9]{7}', str(row['Desc2']))
        id =  id[0].replace("10000", "")
        bank_statement_df.at[index, 'Transaction Id'] = id
    elif "IntS" in row['Desc3']:
        id_match = re.search(r'IntS[0-9]{12}', row['Desc3'])
        if id_match:
            id = id_match.group(0)
            bank_statement_df.at[index, 'Transaction Id'] = id

In [30]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id
0,1,946402601,500.00,CR,9866468505,100003988014,453362,2023-08-29T09:50:00,NaN,3988014
1,2,946402601,-67.83,DR,shrawancommi-080,shrawancommi-080,IntS100000029254,2023-08-29T17:28:00,NaN,IntS100000029254
2,3,946402601,-4.42,DR,asharcommi2080(16th,asharcommi2080(16th,IntS100000029253,2023-08-29T17:27:00,NaN,IntS100000029253


In [31]:
def standard_format_phase_4(row):
        """
        Determine transaction mode and amount from bank statement data.

        Parameters:
            row (Series): A row of bank statement data.

        Returns:
            Series: Mode (CR/DR) and amount.
        """
        if row['Txn Type'] == 'DR' :
            mode = 'DR'
            amount = abs(row['Amount'])
        elif row['Txn Type'] == 'CR':
            mode = 'CR'
            amount = row['Amount']
        else:
            mode = None
            amount = None
        return pd.Series([mode, amount], index=['Mode', 'Amount'])

In [32]:
bank_statement_df[['Mode', 'Amount']] = bank_statement_df.apply(standard_format_phase_4, axis=1)


In [33]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id,Mode
0,1,946402601,500.00,CR,9866468505,100003988014,453362,2023-08-29T09:50:00,NaN,3988014,CR
1,2,946402601,67.83,DR,shrawancommi-080,shrawancommi-080,IntS100000029254,2023-08-29T17:28:00,NaN,IntS100000029254,DR
2,3,946402601,4.42,DR,asharcommi2080(16th,asharcommi2080(16th,IntS100000029253,2023-08-29T17:27:00,NaN,IntS100000029253,DR


In [34]:
bank_statement_df['Date'] = pd.to_datetime(bank_statement_df['Date'], format="%Y-%m-%dT%H:%M:%S").dt.date

In [35]:
bank_statement_df

,S.N,Ac.Number,Amount,Txn Type,Desc1,Desc2,Desc3,Date,Number,Transaction Id,Mode
0,1,946402601,500.00,CR,9866468505,100003988014,453362,2023-08-29,NaN,3988014,CR
1,2,946402601,67.83,DR,shrawancommi-080,shrawancommi-080,IntS100000029254,2023-08-29,NaN,IntS100000029254,DR
2,3,946402601,4.42,DR,asharcommi2080(16th,asharcommi2080(16th,IntS100000029253,2023-08-29,NaN,IntS100000029253,DR
